In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from keras.callbacks import EarlyStopping
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from src import camvid
from src import segnet
from src.callbacks import PlotMetrics
from src.utils import history_to_results
from src.utils import heatmap
from src.wrappers import MonteCarlo
%matplotlib inline
plt.style.use('dark_background')

In [ ]:
!mkdir -p models
weights_file = 'models/BayesianSegNet-CamVid11.h5'

In [ ]:
mapping = pd.read_table('11_class.txt', sep=r'\s+', names=['og', 'new'], index_col='og')['new'].to_dict()
mapping

In [ ]:
batch_size = 8
# the size to reshape images to before transformation
target_size = (360, 480)
# the size to crop images to
crop_size = (352, 480)

In [ ]:
# all crop dimensions must be divisible by this value due 
# to the requirement of equal shapes between downsampling 
# outputs and upsampling inputs imposed by the concatenation
# in skip link connections
divisible_by = int(2**5)
# iterate over all the crop dimensions
for dim in crop_size:
    # raise error if the dimension has a remainder when divided
    if dim % divisible_by:
        f = 'crop dimension ({}) must be divisible by {}'
        f = f.format(dim, divisible_by)
        raise ValueError(f)

# Training

In [ ]:
camvid11 = camvid.CamVid(
    mapping=mapping, 
    target_size=target_size, 
    crop_size=crop_size, 
    batch_size=batch_size,
    ignored_labels=['Void'],
)
generators = camvid11.generators()

In [ ]:
# get the next X, y training tuple
X, y = next(generators['train'])
# transform the onehot vector to an image
y = camvid11.unmap(y)
# plot the images
camvid.plot(X=X[0], y=y[0], order=['X', 'y'])

In [ ]:
# build the model for the image shape and number of labels
model = segnet.build_segnet((*crop_size, 3), camvid11.n, 
    label_names=camvid11.discrete_to_label_map,
    class_weights=camvid11.class_weights,
    dropout_rate=0.5,
)
model.summary()

In [ ]:
# create the callbacks for the training procedure
callbacks = [
    EarlyStopping(monitor='loss', patience=50),
    LearningRateScheduler(lambda _, lr: 0.95 * lr),
    ModelCheckpoint(weights_file, 
        monitor='val_categorical_accuracy', 
        save_best_only=True, 
        save_weights_only=True, 
        mode='max'
    ),
    PlotMetrics(),
]

In [ ]:
# fit the model with the data. divide the steps per epoch by the 
# batch size (which is 3 in this case)
history = model.fit_generator(generators['train'],
    epochs=200,
    steps_per_epoch=int(367 / batch_size),
    validation_data=generators['val'],
    validation_steps=101,
    callbacks=callbacks,
    verbose=0
)

In [ ]:
history_to_results(history)

# Test Metrics

In [ ]:
model.load_weights(weights_file)

In [ ]:
# wrap the model with the test time Monte Carlo
model = MonteCarlo(model, 40)

In [ ]:
metrics = model.evaluate_generator(generators['test'], steps=233)
names = model.metrics_names
pd.DataFrame(metrics, names, columns=['test'])

# Qualitative Results

In [ ]:
def predict(generator) -> tuple:
    """
    Return post-processed predictions for the given generator.

    Args:
        generator: the generator to get data from

    Returns:
        a tuple of for NumPy tensors with RGB data:
        - the batch of RGB X values
        - the unmapped RGB batch of y values
        - the unmapped RGB predicted mean values from the model
        - the meatmap RGB values of the model variance

    """
    # get the batch of data
    X, y = next(generator)
    # predict mean values and variance
    u, s2 = model.predict(X)
    # calculate the mean variance over the labels
    s2 = s2.mean(axis=-1)
    # return X values, unmapped y and u values, and heatmap of s2
    return X, camvid11.unmap(y), camvid11.unmap(u), heatmap(1 - s2, 'bone')

### Train Results

In [ ]:
X, y, p, s = predict(generators['train'])

In [ ]:
camvid.plot(X=X[0], y=y[0], y_pred=p[0], uncertainty=s[0], order=['X', 'y', 'y_pred', 'uncertainty'])

In [ ]:
camvid.plot(X=X[1], y=y[1], y_pred=p[1], uncertainty=s[1], order=['X', 'y', 'y_pred', 'uncertainty'])

In [ ]:
camvid.plot(X=X[2], y=y[2], y_pred=p[2], uncertainty=s[2], order=['X', 'y', 'y_pred', 'uncertainty'])

In [ ]:
camvid.plot(X=X[3], y=y[3], y_pred=p[3], uncertainty=s[3], order=['X', 'y', 'y_pred', 'uncertainty'])

### Validation Results

In [ ]:
X, y, p, s = predict(generators['val'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], uncertainty=s[0], order=['X', 'y', 'y_pred', 'uncertainty'])

In [ ]:
X, y, p, s = predict(generators['val'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], uncertainty=s[0], order=['X', 'y', 'y_pred', 'uncertainty'])

In [ ]:
X, y, p, s = predict(generators['val'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], uncertainty=s[0], order=['X', 'y', 'y_pred', 'uncertainty'])

In [ ]:
X, y, p, s = predict(generators['val'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], uncertainty=s[0], order=['X', 'y', 'y_pred', 'uncertainty'])

### Test Results

In [ ]:
X, y, p, s = predict(generators['test'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], uncertainty=s[0], order=['X', 'y', 'y_pred', 'uncertainty'])

In [ ]:
X, y, p, s = predict(generators['test'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], uncertainty=s[0], order=['X', 'y', 'y_pred', 'uncertainty'])

In [ ]:
X, y, p, s = predict(generators['test'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], uncertainty=s[0], order=['X', 'y', 'y_pred', 'uncertainty'])

In [ ]:
X, y, p, s = predict(generators['test'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], uncertainty=s[0], order=['X', 'y', 'y_pred', 'uncertainty'])